In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import minmax_scale, scale
from tqdm import tqdm
import time
import implicit
from collections import Counter
from surprise import Dataset, Reader
from surprise import SVD
import pickle

c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\track_artists.csv"
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\ensemble_bpr_artist_svd_625_075_ss"
bpr_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\final_bpr_model_backup"
svd_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\final_svd_model_backup"

In [3]:
%%time

list_coord = []
idx = 0

with open(train_path) as f:
    lines = f.readlines()
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        for track in tracks:
            list_coord.append((idx, track))
        idx += 1
        
first_id_test = idx

with open(test_path) as f:
    lines = f.readlines()
    for line in tqdm(lines):
        tracks = line.strip().split(' ')        
        for track in tracks:
            list_coord.append((idx, track))
        idx += 1
        
last_id_test = idx - 1

100%|███████████████████████████████████████████████████████████████████████| 289914/289914 [00:04<00:00, 66145.88it/s]

CPU times: total: 16.8 s
Wall time: 16.8 s


In [4]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)

CPU times: total: 1min 3s
Wall time: 1min 3s


In [5]:
del arr_rows
del arr_cols
del ones
del list_coord

In [6]:
model = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                 learning_rate=0.075,
                                                 regularization=0.0025,
                                                 iterations=950,
                                                 verify_negative_samples=True,
                                                 random_state=42)   
print('model_bpr.fit:')
model.fit(user_item)

model_bpr.fit:


100%|████████████████████████████████████████████| 950/950 [10:20:47<00:00, 39.21s/it, train_auc=99.03%, skipped=0.78%]


In [7]:
%%time

pickle.dump(model, open(bpr_path, "wb"))

CPU times: total: 1.58 s
Wall time: 2.38 s


In [8]:
first_id_test, last_id_test

(1160084, 1449997)